In [1]:
# hide
# default_exp core.data_conversion
import os
from nbdev.showdoc import *
if not os.path.exists('settings.ini'):
    os.chdir('..')

# Data Conversion

> Convert data format to fit the needs of a given component. Contrary to Transforms, the DataConverter classes only change the format of the data, not the data iself.

## DataConverter

In [2]:
#export
import abc
import pandas as pd
import numpy as np

#block-types
from block_types.config import bt_defaults as dflt
from block_types.utils.utils import set_logger

class DataConverter ():
    """
    Convert input and output data format.
       
    This class allows to convert the format of the data before fitting 
    and before transforming, and revert the changes back after performing 
    these operations. This allows to decouple the implementation of a 
    particular component from the remaining components in the pipeline, 
    making it more reusable across different pipelines.
    """
    def __init__ (self, logger = None, verbose: int = dflt.verbose, **kwargs):
        """
        Initialize common attributes and fields, in particular the logger.
        
        Parameters
        ----------
        logger : logging.Logger or None, optional
            Logger used to write messages
        verbose : int, optional
            Verbosity, 0: warning or critical, 1: info, 2: debug.
        """
        # logger used to display messages
        if logger is None:
            self.logger = set_logger ('block_types', verbose=verbose)
        else:
            self.logger = logger
    
    def convert_before_fitting (self, X, y=None):
        """
        Convert incoming data before running fit method.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used for fitting model parameters
        y : labels (N observations), optional
            One dimensional array with N groundtruth labels.
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        y : labels (N observations)
            labels with transformed format but same content 
        """
        return X, y
    
    def convert_after_fitting (self, X):
        """
        Convert data after running fit method.
        
        Calling this method is only required when convert_before_fitting 
        changes X "in place", instead of changing a copy of X. This might 
        be more efficient sometimes, and we have convert_after_fitting to 
        revert the previous change.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used for fitting model parameters
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        """
        return X
    
    def convert_before_transforming (self, X, **kwargs):
        """
        Convert data before running transform method.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used to be transformed
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        """
        return X
    
    def convert_after_transforming (self, result, **kwargs):
        """
        Convert result obtained after by transform method.
        
        Parameters
        ----------
        result : data (N' observations x D' dimensions)
                result obtained by transformed method
            
        Returns
        -------
        result : data (N' observations x D' dimensions)
            result with transformed format but same content
        """
        return result

## NoConverter

In [3]:
#export
class NoConverter (DataConverter):
    """Performs no conversion."""
    pass

In [4]:
dc = NoConverter()
X, y = [1,2,3], [0,1,0]
Xr, yr = dc.convert_before_fitting (X, y)
assert X==Xr
assert y==yr

## PandasConverter

In [5]:
#export
class PandasConverter (DataConverter):
    """
    Convert DataFrame to numpy array and back, if needed.
        
    By default, this class assumes the following:

        - When calling the fit method, the data is received 
          as a DataFrame. This DataFrame contains not only the 
          data to be used for fitting our model, but also the 
          ground-truth labels. The `PandasConverter` takes only 
          the data needed for fitting the model, and puts it 
          into a matrix `X`, and then takes the labels and puts 
          them into a separate vector `y`. While all this is done 
          by default, the `PandasConverter` also allows other 
          possibilities: receiving the data and the labels separately 
          in `X` and `y`, in which case no action is needed, or avoiding 
          to separate the data and the labels (if the flag `separate_labels` is False), 
          in which case the matrix `X` will contain both data and labels. 
          It also allows to receive numpy arrays instead of DataFrames,
          in which case the data format is preserved.

        - When calling the `transform` method, the `PandasConverter` 
          removes by default the labels from the incoming DataFrame, 
          and then puts them back after performing the transformation. 
          This behaviour can change if we set `transform_needs_labels=True`. 
          In this case, the labels are kept in the matrix `X` so that 
          they can be used during the transformation. This is done in 
          particular by one type of component called `SamplingComponent`, 
          defined in `core.component_types`. This is useful for 
          components that do some sort of under-sampling or over-sampling, 
          changing the number of observations. When this occurs, the 
          labels need to be adjusted accordingly, so that the `transform` 
          method modifies both the data and the labels, both of whom are 
          contained in the output matrix `X`.

        The  default `DataConverter` used in the current implementation is the 
        `PandasConverter`. 

        #### Note on generic use of metadata (to be implemented)

        In general, our DataFrames behave like a single-table in-memory DataBases 
        from which we can take the necessary data and metadata to perform any 
        operation needed in our pipeline. Although currently we only consider 
        groundtruth labels as metadata, in the future we plan to allow any other 
        metadata indicated by configuration. This includes the `chiller_id`, which 
        might be needed by some of the components, to differentiate between the data of 
        different chillers, for data-sets with more than one chiller. Currently 
        our dataset contains a single chiller, and this type of metadata is not needed. 
        Regardless of the metadata being used, the `PandasConverter` takes only the data 
        needed for fitting the model, puts it into a matrix `X`, and then takes the 
        labels and puts them into a separate vector `y`. The rest of the metadata is 
        discarded unless the component needs it for some purpose, in which case this 
        will be indicated by a parameter called something like `metadata`, which contains 
        the list of columns in the DataFrame which contain the rest of metadata. 
    """
    def __init__ (self,
                  transform_uses_labels=False,
                  transformed_index=None,
                  transformed_columns=None,
                  separate_labels=True,
                  **kwargs):
        """
        Initialize attributes and fields.
        
        Parameters
        ----------
        transform_uses_labels : bool, optional
            If True, the `transform` method receives as input data `X` a DataFrame where 
            one of the columns is `label`, containing the ground-truth labels. This allows 
            the transform method to modify the number of observations, 
            changing the number of rows in the data and in the labels.
            See `SamplingComponent` class in `block_types.core.component_types`.
            If False, the input data `X` only contains data consumed by , without 
            ground-truth labels. 
        transformed_index : array-like or None, optional
            Used after transforming the data. If the result of the transformation is 
            a numpy array, two things can happen: 1) if the number of rows of this array 
            is the same as the number of rows of the input DataFrame, then we convert 
            the array to a DataFrame with the same index as the original; 2) if the number
            of rows is not the same, the index used for the new DataFrame is 
            `transformed_index` if provided, or 0..N-1 (where N=number of rows) if not 
            provided. 
        transformed_columns : array-like or None, optional
            Used after transforming the data. If the result of the transformation is 
            a numpy array, two things can happen: 1) if the number of columns of this array 
            is the same as the number of columns of the input DataFrame, then we convert 
            the array to a DataFrame with the same columns as the original; 2) if the number
            of columns is not the same, the columns used for the new DataFrame is 
            `transformed_columns` if provided, or 0..D-1 (where D=number of columns) if not 
            provided. 
        separate_labels : bool, optional
            Used before calling the fit method. If separate_labels=True (default value), 
            the `fit` method receives the data and labels separately in `X` and `y` 
            respectively. If separate_labels=False, the `fit` method receives both the 
            data and the labels in the same input `X`, where the labels are in a 
            column of `X` called `label` (TODO: make this configurable). This last 
            option is used by the `Pipeline` class, and its rationale is provided in 
            the description of that class.
        """

        super().__init__(**kwargs)
        
        # whether the _transform method receives a DataFrame that includes the labels, or it doesn't
        self.transform_uses_labels = transform_uses_labels

        # configuration for converting the transformed data into a DataFrame
        self.transformed_index = transformed_index
        self.transformed_columns = transformed_columns
        
        # whether the _fit method receives a DataFrame that includes the labels, or the labels are placed separately in y
        self.separate_labels = separate_labels

    def convert_before_fitting (self, X, y=None):
        """
        By default, convert DataFrame X to numpy arrays X and y
        
        The most common use of this method is:
        - When calling the fit method, the data is received 
          as a DataFrame. 
        - This DataFrame contains not only the data to be 
          used for fitting our model, but also the 
          ground-truth labels. This method takes only 
          the data needed for fitting the model, and puts it 
          into a matrix `X`, and then takes the labels and puts 
          them into a separate vector `y`. 
        
        Other possibilities are:
          - If the data and the labels are separated in `X` and `y`  
          (i.e., X does not include labels), no action is performed.
          - If `self.separate_labels` is False, the data and the labels 
          are not separated, in which case the data `X`
          passed to the fit method will contain both data and labels. 
          - It also allows to receive numpy arrays instead of DataFrames,
          in which case the data format is preserved.
        """
        if self.separate_labels and (type(X) is pd.DataFrame) and ('label' in X.columns):
            if y is None:
                y = X['label']
            else:
                assert (y==X['label']).all(), "discrepancy between y and X['label']"
                
            X = X.drop(columns='label')
            self.restore_label_fitting = True
            self.y_fitting = y
        else:
            self.restore_label_fitting = False

        return X, y

    def convert_after_fitting (self, X):
        """Do nothing. Return same data received."""
        return X

    def convert_before_transforming (self, X, new_columns=None, **kwargs):
        """
        By default, remove labels from incoming DataFrame.
        
        This method allows to remove the labels from the incoming DataFrame, 
        and then put them back after performing the transformation. 
        This behaviour can change if we set `self.transform_needs_labels=True`. 
        In this case, the labels are kept in the matrix `X` so that they can be 
        used during the transformation. This is done in particular by one type of 
        component called `SamplingComponent`, defined in `core.component_types`. 
        This is useful for components that do some sort of under-sampling or 
        over-sampling, changing the number of observations. When this occurs, 
        the labels need to be adjusted accordingly, so that the `transform` method 
        modifies both the data and the labels, both of whom are contained in the output 
        matrix `X`.
        
        
        """
        if new_columns is None:
            new_columns = self.transformed_columns
        self.new_columns = new_columns
        if (type(X) is pd.DataFrame) and ('label' in X.columns) and (not self.transform_uses_labels):
            y = X['label']
            X = X.drop(columns='label')
            self.restore_label_transform = True
            self.y_transform = y
        else:
            self.restore_label_transform = False

        self.type_X = type(X)
        if self.type_X is pd.DataFrame:
            self.X_shape = X.shape
            self.X_index = X.index
            self.X_columns = X.columns
            if 'label' in self.X_columns:
                self.X_label = X['label']
        return X

    def convert_after_transforming (self, result, **kwargs):
        """
        Convert the result produced by `transform`to DataFrame format.
        
        If the input to `transform` was in DataFrame format, the `result` 
        given by `transform` is converted to DataFrame if it is not 
        produced in this format. Furthermore, if the `label` column was 
        in the input to `transform` and it is not in the output given
        by `transform`, it is appended to the result.
        """

        result = self.convert_to_dataframe (result)
        if self.restore_label_transform:
            if type(result) is pd.DataFrame:
                if 'label' in result.columns:
                    self.logger.warning ('label already part of result')
                result['label'] = self.y_transform
            else:
                self.logger.warning ('result is not DataFrame')

        return result

    def convert_to_dataframe (self, result):
        """Convert the `result` produced by `transform`to DataFrame format."""
        
        if self.type_X is pd.DataFrame:
            if type(result) is np.ndarray:
                if result.shape[0] == self.X_shape[0]:
                    index = self.X_index
                else:
                    index = self.transformed_index if (self.transformed_index is None) else range(result.shape[0])
                if (result.ndim > 1) and (result.shape[1] == self.X_shape[1]):
                    columns = self.X_columns
                else:
                    columns = self.new_columns if (self.new_columns is not None) else range(result.shape[1]) if (result.ndim > 1) else [0]

                result = pd.DataFrame (result, index=index, columns=columns)
            elif (type(result) is pd.DataFrame and self.new_columns is not None and 
                  result.shape[1]==len(self.new_columns) and not (result.columns==self.new_columns).all()):
                result.columns = self.new_columns

            if type(result) is pd.DataFrame:
                if ('label' in self.X_columns) and ('label' not in result.columns):
                    self.logger.info ('label column not found in result, but found in input DataFrame')
                    result['label'] = self.X_label
        return result

## Window2Dto3Dconverter

In [6]:
#export
class Window2Dto3Dconverter (DataConverter):
    """Convert sequence of windows from WindowGenerator's 2D format to 3D. 
    
    Given a 2D Dataframe of size N x (W*D), where N=number of windows,
    D=number of variables (dimensions), and W=size of windows, converts this 
    to a numpy array of N x D x W. Note that the order of the elements is 
    transposed: for each window, the has first the elements of a window in 
    one dimension, then the elements in the second dimension, etc. This 
    is transposed in the output to have the second and third axis be D and W 
    respectively.
    """
    def __init__ (self, sequence_length: int, data_converter: DataConverter = None, **kwargs):
        """
        Initialize common attributes and fields.
        
        Parameters
        ----------
        sequence_length : int
            Size of each window.
        data_converter : DataConverter, optional
            DataConverter that will transform the input data to a 2D DataFrame of 
            size N x (D*W), if it is not already in this format. PandasConverter 
            is used by default.
        """
        self.sequence_length = sequence_length
        if data_converter is None:
            self.data_converter = PandasConverter (**kwargs)
        else:
            self.data_converter = data_converter

    def convert_before_fitting (self, X, y=None):
        """
        Convert incoming data before running fit method.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used for fitting model parameters
        y : labels (N observations), optional
            One dimensional array with N groundtruth labels.
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        y : labels (N observations)
            labels with transformed format but same content 
        """
        X, y = self.data_converter.convert_before_fitting (X, y)
        X = self.transform (X)
        return X, y

    def convert_after_fitting (self, X):
        """
        Convert data after running fit method.
        
        Calling this method is only required when convert_before_fitting 
        changes X "in place", instead of changing a copy of X. This might 
        be more efficient sometimes, and we have convert_after_fitting to 
        revert the previous change.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used for fitting model parameters
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        """
        return self.data_converter.convert_after_fitting (X)

    def convert_before_transforming (self, X, **kwargs):
        """
        Convert data before running transform method.
        
        Parameters
        ----------
        X : data (N observations x D dimensions)
            data used to be transformed
            
        Returns
        -------
        X : data (N observations x D dimensions)
            data with transformed format but same content
        """
        X = self.data_converter.convert_before_transforming (X, **kwargs)
        X = self.transform (X)
        return X

    def convert_after_transforming (self, result, **kwargs):
        """
        Convert result obtained after by transform method.
        
        Parameters
        ----------
        result : data (N' observations x D' dimensions)
                result obtained by transformed method
            
        Returns
        -------
        result : data (N' observations x D' dimensions)
            result with transformed format but same content
        """
        result = self.inverse_transform (result)
        result = self.data_converter.convert_after_transforming (result, **kwargs)
        return result

    def transform (self, df):
        """
        Convert input DataFrame `df` to numpy array in 3D format.
        
        Given a 2D Dataframe of size N x (W*D), where N=number of windows,
        D=number of variables (dimensions), and W=size of windows, converts this 
        to a numpy array of N x D x W. Note that the order of the elements is 
        transposed: for each window, the has first the elements of a window in 
        one dimension, then the elements in the second dimension, etc. This 
        is transposed in the output to have the second and third axis be D and W 
        respectively.
        """
        data = df.values.reshape(df.shape[0], -1, self.sequence_length)
        data = np.transpose(data, (0,2,1))
        return data

    def inverse_transform (self, data):
        """
        Convert 3D numpy array `data` to 2D DataFrame. 
        
        Given a 3D numpy array of size N x D x W, convert this to Dataframe 
        of size N x (W*D), where N=number of windows, D=number of variables 
        (dimensions), and W=size of windows. Note that the order of the 
        elements is transposed: for each window, the output provides first 
        the elements of the first variable across the time-steps of the window, 
        then the elements of the second variable, etc.
        """
        data = np.transpose (data, (0,2,1))
        data = data.reshape (data.shape[0], -1)
        return data

## Factory

In [7]:
#export
def data_converter_factory (converter, 
                            *args, 
                            **kwargs):
    if type(converter) is str:
        cls = eval(converter)
    elif type(converter) is type:
        cls = converter
    elif isinstance (converter, DataConverter):
        return converter
    else:
        raise ValueError (f'invalid converter {converter}, must be str, class or object instance of DataConverter')
    return cls(*args, **kwargs)

In [10]:
converter = data_converter_factory ('PandasConverter', separate_labels=False)
assert type(converter) is PandasConverter
assert converter.separate_labels is False